In [1]:
import os

In [2]:
pwd

'c:\\Users\\junu\\OneDrive\\Desktop\\ds\\MLOPS\\media_campign_cost_prediction\\branches\\basic-set-up\\Media-Campign-cost-prediction\\research'

In [3]:
os.chdir('../')
# os.chdir('Media-Campign-cost-prediction')

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass 
class ModelEvaluationConfig:
    model_evaluation_root: Path
    model_path: Path
    evaluation_score: Path
    test_data_path: Path
    target: str
    params: dict

In [5]:
from src.constants import *
from src.utils.common import read_yaml,create_directories,save_model

class Configuration:
    def __init__(self,config_path=CONFIG_FILE_PATH
                      ,params_path=PARAMS_FILE_PATH
                      ,schema_path=SCHEMA_FILE_PATH) -> None:
        self.config=read_yaml(config_path)
        self.params=read_yaml(params_path)
        self.schema=read_yaml(schema_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self):
        config= self.config.model_evaluation
        model_eval_config=ModelEvaluationConfig(
            model_evaluation_root=Path(config.model_evaluation_root),
            model_path=Path(config.model_path),
            evaluation_score=Path(config.evaluation_score),
            test_data_path=Path(config.test_data_path),
            target= config.target,
            params= dict(self.params.xgboost)

        )
        return model_eval_config


In [7]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/junaisk456/Media-Campign-cost-prediction.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='junaisk456'
os.environ['MLFLOW_TRACKING_PASSWORD']='a4dcf1ebabde463c427fa51ac0272a6b4b0341e6'

In [8]:
from src.utils.common import load_model,create_directories,save_json
import pandas as pd
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,mean_squared_log_error
from src import logger
import mlflow
from mlflow.models.signature import infer_signature

class ModelEvaluation:
    def __init__(self,config: ModelEvaluationConfig) -> None:
        self.config=config
        
    def initiate_evaluation(self):
        model=load_model(self.config.model_path)
        eval_data=pd.read_csv(self.config.test_data_path)
        X=eval_data.drop(self.config.target,axis=1)
        y_test=eval_data[self.config.target]
        logger.info(self.config.params)
        params=self.config.params
        #model prediction
        y_pred=model.predict(X)
        create_directories([self.config.model_evaluation_root])
        mae=mean_absolute_error(y_test,y_pred)
        rmse=mean_squared_error(y_test,y_pred,squared=False)
        r2=r2_score(y_test,y_pred)
        rmlse=mean_squared_log_error(y_test,y_pred,squared=False)
        logger.info(f" model - \n MAE = {mae} \n RMSE = {rmse} \n R2 Score = {r2} \n RMLSE = {rmlse} \n ============================")

        metrics={"MAE":mae,
               "RMSE":rmse,
               "r2":r2,
               "rmlse":rmlse}
        
        eval_score={
            "Model Parms":params,
            "evaluation metrics":metrics

        }
        save_json(self.config.evaluation_score,eval_score)
        # Set our tracking server uri for logging
        mlflow.set_tracking_uri(uri="https://dagshub.com/junaisk456/Media-Campign-cost-prediction.mlflow")

# Create a new MLflow Experiment
        mlflow.set_experiment("Media Cost prediction Tracker")

# Start an MLflow run
        with mlflow.start_run():
    # Log the hyperparameters
            mlflow.log_params(params)

    # Log the loss metric
            mlflow.log_metrics(metrics)

    # Set a tag that we can use to remind ourselves what this run was for
            mlflow.set_tag("Training Info", "Xg boost model")

    # Infer the model signature
            signature = infer_signature(model_input=X,model_output=y_pred)

            # Log the model
            model_info = mlflow.xgboost.log_model(
                xgb_model=model,
                artifact_path="artifacts",
                signature=signature,
                input_example=X,
                registered_model_name="XGBoost model1",
            )


    
 


In [9]:
config=Configuration()
eval_config=config.get_model_evaluation_config()
eval=ModelEvaluation(eval_config)
eval.initiate_evaluation()

[2024-04-04 23:11:08,907: INFO: 19 : call: file loaded succesfully from the path config\config.yaml]
[2024-04-04 23:11:08,928: INFO: 19 : call: file loaded succesfully from the path params.yaml]
[2024-04-04 23:11:08,941: INFO: 19 : call: file loaded succesfully from the path schema.yaml]


[2024-04-04 23:11:09,261: INFO: 17 : call: {'objective': 'reg:squarederror', 'booster': 'gbtree', 'learning_rate': 0.3, 'max_depth': 6, 'alpha': 8, 'n_estimators': 40, 'gamma': 3, 'verbosity': 2, 'min_child_weight': 2, 'reg_lambda': 1, 'scale_pos_weight': 4, 'eval_metric': 'rmse'}]
[2024-04-04 23:11:09,387: INFO: 26 : call:  model - 
 MAE = 24.25746600866434 
 RMSE = 28.374718588914945 
 R2 Score = 0.10173379007128436 
 RMLSE = 0.30340002914546205 
 ============================]
[2024-04-04 23:11:09,390: INFO: 38 : call: json file save successfully at artifacts\model_evaluation\score.json]


2024/04/04 23:11:10 INFO mlflow.tracking.fluent: Experiment with name 'Media Cost prediction Tracker' does not exist. Creating a new experiment.
c:\Users\junu\anaconda3\envs\media\lib\site-packages\mlflow\models\signature.py:130: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  inputs = _infer_schema(model_input)
c:\Users\junu\anaconda3\envs\media\lib\site-packages\xgboost\cor

In [ ]:
eval_metrcis={
    "name":"j u",
    "age":25
}
path=

save_json()

In [46]:
!mlflow server --host 127.0.0.1 --port 8080

^C


In [ ]:
MLFLOW_TRACKING_URI=https://dagshub.com/junaisk456/Media-Campign-cost-prediction.mlflow \
MLFLOW_TRACKING_USERNAME=junaisk456 \
MLFLOW_TRACKING_PASSWORD=a4dcf1ebabde463c427fa51ac0272a6b4b0341e6 \
python script.py

In [16]:
os.env['MLFLOW_TRACKING_URI']='https://dagshub.com/junaisk456/Media-Campign-cost-prediction.mlflow'

AttributeError: module 'os' has no attribute 'env'